## Model evaluation from saved model (trained in google Colab)

In [1]:
from keras.models import model_from_json

model_dir = "D:\\tamu\\courses\\DeepLearning\\ProjectPart4\\results\\models"


json_file = open(model_dir +'\\model_CNN_json.json')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(model_dir + '\\model_CNN_weight.h5')
print("Loaded model from disk")

Using TensorFlow backend.


Loaded model from disk


In [2]:
from keras.applications import VGG16

conv_base = VGG16(weights= 'imagenet', include_top=False, input_shape=(250, 250, 3))

conv_base.trainable = False

In [3]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
#model.summary()

In [4]:
from keras import optimizers

model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['accuracy'])

In [5]:
test_img_dir = "D:\\tamu\\courses\\DeepLearning\\ProjectPart4\\test_img"

import os

test_label =[]
for file in os.listdir(test_img_dir):
    label = file.split('_')[0]
    if label == 'other':
        test_label.append(0)
    else:
        test_label.append(1)
print (len(test_label))

200


In [6]:
import glob
import cv2  
import numpy as np

def get_Xtrain(file_name):
    X_train = []
    for i , filename in enumerate(glob.glob(file_name)):
        image = cv2.imread(filename)
        res_img = cv2.resize(image, dsize=(250,250), interpolation=cv2.INTER_CUBIC)
        X_train.append(res_img)
    
    return X_train 

In [7]:
X_t = get_Xtrain(test_img_dir + '\\*.jpg')

x_test = np.asarray(X_t)
y_test = np.asarray(test_label)

print('x_test.shape: ', x_test.shape)
print('y_test.shape: ', y_test.shape)

x_test = x_test/x_test.max()

x_test.shape:  (200, 250, 250, 3)
y_test.shape:  (200,)


In [8]:
scores = model.evaluate(x_test, y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

200/200 [==============================] - 56s 280ms/step
accuracy: 40.00%


In [10]:
predictions = model.predict(x= x_test)